<a href="https://colab.research.google.com/github/GabrielFCosta/TCC-BSI-2023.1/blob/main/old/matc97_gabriel_teste_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import string
import nltk
import math
import random
import pandas as pd
import time as tm
from urllib.request import urlopen
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
# Remove pontuação, utilizada pela função seguinte de pré-processamento
def remover_pontuacao(tokens):
  for token in tokens:
    if token in string.punctuation:
      tokens.remove(token)
# Função de pré-processamento
def preprocess(text):
    text = word_tokenize(text)
    remover_pontuacao(text)
    return pos_tag(text)

# Listas para coletar rótulos e entidades (palavras) do texto
codes = []
words = []
def  clearlists():
  if len(words) > 0:
    words.clear()
  if len(codes) > 0:
    codes.clear()

# Função de reconhecimento de entidades nomeadas (NER)
def ner(sent):
  ne_tree = nltk.ne_chunk(sent)
  for chunk in ne_tree:
    if hasattr(chunk,"label"):
      word =' '.join(c[0] for c in chunk)
      #print(chunk.label(),word)
      codes.append(chunk.label())
      words.append(word)

# Função que retorna dataframe de entidades (palavras), rótulos e frequências 
def returndataset(col1,col2):
  # junta listas de palavras e rótulos num dataframe
  dataset = pd.DataFrame(zip(words,codes),columns =[col1,col2])
  clearlists()
  # calcula frequências das palavras e adiciona nova coluna
  dataset['freq'] = dataset.groupby([col1,col2])[col1].transform('count')
  return dataclean(dataset)
 
# Remove linhas duplicatas e atualiza index
def dataclean(dataset):
  dataset.drop_duplicates(inplace= True)
  dataset.reset_index()
  return dataset

# Verifica rótulo gramatical
def test(tag):
  # verificando todos substantivos, adjetivos, verbos e numerais cardinais.
  #tags = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ','CD']
  tags = ['NN','NNS','NNP','NNPS']
  for i in tags:
    if i == tag:
      return True
  return False

# Filtra palavras da lista gerada pelo pos tagging e retorna dataframe ou lista de palavras
def filtertuples(textlist,rtdata):
  for tuple in textlist:
    if test(tuple[1]):
      words.append(tuple[0])
      codes.append(tuple[1])
  if rtdata:
    return returndataset('word','pos_tag')
  else:
    aux = words.copy()
    clearlists()
    return aux

# Retorna união de dois dataframes com "frequência de documentos" 
def datamerge(words0,words1,col1,col2):
  merged = pd.merge(words0,words1, on=[col1,col2], how='outer')
  merged = dataclean(merged).fillna(0)
  merged['doc_freq'] = getdoc_freq(merged)
  return merged.sort_values(by=[col1,col2])

# Retorna lista de "frequência de documentos"
def getdoc_freq(merged):
  doc_freq = []
  c = 0
  # verifica duas colunas de frequência
  for i in merged['freq_x'].tolist():
    # se frequência maior que 0 em ambas colunas 
    if i > 0 and merged['freq_y'].tolist()[c] > 0:
      doc_freq.append(2)
    else:
      doc_freq.append(1)
    c += 1
  return doc_freq

# interseção dividida pela união com base na frequência de documentos
def jaccard_simples(doc_freq):
    intersecao = 0
    c = 0
    for x in doc_freq:
      if x == 2:
        intersecao += 1
      c += 1
    #print("interseção:",intersecao)
    uniao = len(doc_freq)
    #print("união:",uniao)
    return intersecao/uniao

# Não penderado, calculado a partir das frequências
# De acordo com https://towardsdatascience.com/nlp-text-similarity-how-it-works-and-the-math-behind-it-a0fb90a05095
def cosine_simples(mergedf):
  numerador = 0
  vetx = 0
  vety = 0
  for idx, row in mergedf.iterrows():
    numerador += row['freq_x'] * row['freq_y']
    vetx += pow(row['freq_x'],2)
    vety += pow(row['freq_y'],2)
  vetx = math.sqrt(vetx)
  vety = math.sqrt(vety)
  denominador = vetx * vety
  return numerador/denominador

# Similaridade com base na média das similaridades máximas entre palavras
def similaridade_WP_max(str0,str1):
  means = []
  vals = []
  aux = 0
  for wd0 in str0:
    syn0 = wn.synsets(wd0)
    # verifica se 1a palavra contém alguma definição antes de prosseguir
    if len(syn0) != 0:
      vals.clear()
      for wd1 in str1:
        syn1 = wn.synsets(wd1)
        # verifica se 2a palavra contém alguma definição antes de calcular similaridade
        if len(syn1) != 0:
          aux = syn0[0].wup_similarity(syn1[0])
          # se similaridade = 1, palavras iguais, break
          if aux == 1:
            break
          # senão, adiciona valor à lista vals
          else:
            vals.append(aux)
            #print("syn0:",syn0[0],"syn1:",syn1[0],"wup:",syn0[0].wup_similarity(syn1[0]))
      # se aux = 1, similaridade máxima = 1
      if aux == 1:
         means.append(aux)
      # senão seleciona máxima da lista
      else:
        means.append(max(vals))
  #print(means)
  return sum(means)/len(means)


In [ ]:
# Retorna lemmas e hiperônimos das palavras contidas na lista do parâmetro
def getclasses(stringlist):
  lemmas = []
  hypernyms = []
  na = 'N\A'
  for wd in stringlist:
    syn = wn.synsets(wd)
    if len(syn) > 0:
      lemmas.append(syn[0].lemmas()[0].name())
      if len(syn[0].hypernyms()) > 0:
        hypernyms.append(syn[0].hypernyms()[0].name())
      else:
        hypernyms.append(na)
    else:
      lemmas.append(na)
      hypernyms.append(na)
  return pd.DataFrame(zip(stringlist,lemmas,hypernyms),columns =['word','lemma','hypernym'])

# Retorna tabela de frequências do tipo de classe (lemma ou hypernym), passado no parâmetro col
def intersects(df0,df1,col):
  aux = []
  for i in df0.tolist():
    if i != 'N\A':
      aux.append(i)
  df0 = pd.DataFrame(zip(aux),columns =[col])
  df0['freq'] = df0.groupby([col])[col].transform('count')
  #print(df0)
  aux.clear()
  for i in df1.tolist():
    if i != 'N\A':
      aux.append(i)
  df1 = pd.DataFrame(zip(aux),columns =[col])
  df1['freq'] = df1.groupby([col])[col].transform('count')
  #print(df1)
  return mergeclass(df0,df1,col)

# Auxiliar à função intersects pra fazer o merge das tabelas do parâmetro
def mergeclass(df0,df1,col):
  merged = pd.merge(df0,df1, on=[col], how='outer')
  merged = dataclean(merged).fillna(0)
  merged['doc_freq'] = getdoc_freq(merged)
  return merged.sort_values(by=[col])


In [ ]:
# Funções de comparação
def compnomeadas(frase0,frase1,analise):
  ner(preprocess(frase0))
  dt0 = returndataset('entity','label')
  ner(preprocess(frase1))
  dt = datamerge(dt0,returndataset('entity','label'),'entity','label')
  #print(dt)
  if analise == 'j':
    return jaccard_simples(dt['doc_freq'].tolist())
  elif analise == 'c':
    return cosine_simples(dt)
  return None

def comppostags(frase0,frase1,analise):
  str0 = filtertuples(preprocess(frase0),True)
  str1 = filtertuples(preprocess(frase1),True)
  #print("string 0:",str0['word'].tolist())
  #print("string 1:",str1['word'].tolist())
  if analise == 'w':
    return similaridade_WP_max(str0['word'].tolist(),str1['word'].tolist())
  else:
    dt = datamerge(str0,str1,'word','pos_tag')
    if analise == 'j':
      return jaccard_simples(dt['doc_freq'].tolist())
    elif analise == 'c':
      return cosine_simples(dt)
  return None

def compwordnet(frase0,frase1,classe,analise):
  #print(preprocess(frase0))
  #print(filtertuples(preprocess(frase0),False))
  str0 = getclasses(filtertuples(preprocess(frase0),False))
  #print(str0,'\n')
  str1 = getclasses(filtertuples(preprocess(frase1),False))
  #print(str1,'\n')
  df = intersects(str0[classe],str1[classe],classe)
  #print(df)
  if analise == 'j':
    return jaccard_simples(df['doc_freq'].tolist())
  elif analise == 'c':
    return cosine_simples(df)
  return None

def compscikit(frase0,frase1):
  corpus = [frase0,frase1]
  vectorizer = TfidfVectorizer()
  trsfm=vectorizer.fit_transform(corpus)
  return cosine_similarity(trsfm[0:1], trsfm)[0][1]


In [ ]:
# Testes com frases simples
frases = ["Airbnb will shut down its listings in China after two years of lockdowns in the country.",
        "Starting this summer, Airbnb will take down its listings and offers for hosted experiences in China.",
        "International brands, from Apple to Estee Lauder, have warned of the financial impact of the restrictions."]

start = tm.time()
print("\n######## Comparação de Entidades Nomeadas ########")
print("\n## strings 0 e 0 (caso base): ##")
print(compnomeadas(frases[0],frases[0],'c'))
print("\n## strings 0 e 1: ##")
print(compnomeadas(frases[0],frases[1],'c'))
print("\n## strings 0 e 2: ##")
print(compnomeadas(frases[0],frases[2],'c'))
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

start = tm.time()
print("\n######## Comparação de POS TAGS ########")
print("\n## strings 0 e 0 (caso base): ##")
print(comppostags(frases[0],frases[0],'w'))
print("\n## strings 0 e 1: ##")
print(comppostags(frases[0],frases[1],'w'))
print("\n## strings 0 e 2: ##")
print(comppostags(frases[0],frases[2],'w'))
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

start = tm.time()
print("\n######## Comparação de POS TAGS ########")
print("\n## strings 0 e 0 (caso base): ##")
print(comppostags(frases[0],frases[0],'c'))
print("\n## strings 0 e 1: ##")
print(comppostags(frases[0],frases[1],'c'))
print("\n## strings 0 e 2: ##")
print(comppostags(frases[0],frases[2],'c'))
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

# cosine do scikit
start = tm.time()
print('\n######## Cosine do Scikit ########')
print("\n## strings 0 e 1: ##")
corpus = [frases[0],frases[1]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
print("\n## strings 0 e 2: ##")
corpus = [frases[0],frases[2]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
print("\n## strings 0, 1 e 2: ##")
corpus = [frases[0],frases[1],frases[2]]
vectorizer = TfidfVectorizer()
trsfm=vectorizer.fit_transform(corpus)
print(cosine_similarity(trsfm[0:1], trsfm))
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")

start = tm.time()
print('\n######## Comparação de Lemmas do Wordnet ########')
print("\n## strings 0 e 0 (caso base): ##")
print(compwordnet(frases[0],frases[0],'lemma','c'))
print("\n## strings 0 e 1: ##")
print(compwordnet(frases[0],frases[1],'lemma','c'))
print("\n## strings 0 e 2: ##")
print(compwordnet(frases[0],frases[2],'lemma','c'))
end = tm.time()
print("\n######## Tempo de execução:", end-start,"########")



######## Comparação de Entidades Nomeadas ########

## strings 0 e 0 (caso base): ##
0.9999999999999998

## strings 0 e 1: ##
0.4999999999999999

## strings 0 e 2: ##
0.0

######## Tempo de execução: 0.09979915618896484 ########

######## Comparação de POS TAGS ########

## strings 0 e 0 (caso base): ##
1.0

## strings 0 e 1: ##
0.7073015873015873

## strings 0 e 2: ##
0.3337628384687208

######## Tempo de execução: 0.06817936897277832 ########

######## Comparação de POS TAGS ########

## strings 0 e 0 (caso base): ##
1.0000000000000002

## strings 0 e 1: ##
0.5000000000000001

## strings 0 e 2: ##
0.0

######## Tempo de execução: 0.06440162658691406 ########

######## Cosine do Scikit ########

## strings 0 e 1: ##
[[1.         0.31639145]]

## strings 0 e 2: ##
[[1.         0.12254598]]

## strings 0, 1 e 2: ##
[[1.         0.35608377 0.15772405]]

######## Tempo de execução: 0.012107610702514648 ########

######## Comparação de Lemmas do Wordnet ########

## strings 0 e 0 (caso ba

In [ ]:
# 1o. carregamento da base de dados de filmes
urls = ["https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/movies.csv",
        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/ratings.csv",
        "https://raw.githubusercontent.com/GabrielFCosta/preprocessamento/main/tags.csv"]
datasets = []
auxlist =[]
for url in urls:
  df = pd.read_csv(url)
  df[df.notnull()]
  datasets.append(df)
# concatena títulos e gêneros na coluna string
datasets[0]['string'] = datasets[0]['title'] + " " + datasets[0]['genres']
# substitui barras verticais '|' por espaços entre gêneros 
aux =''
for idx1, i in datasets[0].iterrows():
  aux = i['string'].replace('|', " ")
  auxlist.append(aux)
datasets[0]['string'] = auxlist
auxlist.clear()
# concatena tags de usuários
for idx1, i in datasets[0].iterrows():
  df = datasets[2].loc[datasets[2]['movieId'] == i['movieId'] ] 
  aux =''
  for idx2, j in df.iterrows():
    aux = aux +" "+ j['tag']
  #print("movie:"+str(i['movieId'])+" tags:"+aux)
  auxlist.append(aux)
datasets[0]['string'] = datasets[0]['string'] + "" + auxlist


In [ ]:
# retorna dataframe de amostras de filmes aleatórios com rótulos de rating máximo
def sampledata(df,sample,maxrated):
  data = pd.DataFrame()
  data = df.sample(n=sample)
  data = data.astype({'movieId':'int'})
  data["max_rated"] = maxrated
  return data

# junta dois dataframes de 10 filmes de rating 5 com outros 40 aleatórios
# retorna dataframe contendo movieIds e rótulos, ordenado por movieId
def mergesamples(usersample,mids):
  merged = pd.merge(usersample,mids, on=['movieId'], how='outer')
  merged = dataclean(merged).fillna(0)
  merged = merged.astype({'max_rated_x':'int'})
  merged.rename(columns = {'max_rated_x':'max_rated'}, inplace = True)
  merged.drop(['userId', 'rating', 'max_rated_y'], axis=1, inplace=True)
  return merged.sort_values(by=['movieId'])

# Retorna string do filme a partir do movieId
def returnstring(movieId):
  ref = datasets[0][datasets[0]['movieId'] == movieId]
  return ref['string'].iloc[0]

# 3o. Comparação seletiva
def batchcompare(refmovie,testsample,comparison):
  refmovie = returnstring(refmovie)
  samplelist = []
  valores = []
  df  = pd.DataFrame()
  df = testsample.copy(deep=True)
  # coleta todas as strings dos filmes de teste na lista samplelist
  for idx, row in testsample.iterrows():
    samplelist.append(returnstring(row['movieId']))
  # compara de acordo com o parâmetro comparison
  if comparison == 'posw':
    for frase in samplelist:
      valores.append(comppostags(refmovie,frase,'w'))
    df['postag_wp'] = valores
    return df.sort_values(by=['postag_wp'], ascending = False).head(10)
  elif comparison == 'posc':
    for frase in samplelist:
      valores.append(comppostags(refmovie,frase,'c'))
    df['postag_cos'] = valores
    return df.sort_values(by=['postag_cos'], ascending = False).head(10)
  elif comparison == 'wnet':
    for frase in samplelist:
      valores.append(compwordnet(refmovie,frase,'lemma','c'))
    df['wnet_lemma_cos'] = valores
    return df.sort_values(by=['wnet_lemma_cos'], ascending = False).head(10)
  elif comparison == 'scik':
    for frase in samplelist:
      valores.append(compscikit(refmovie,frase))
    df['scikit_cos'] = valores
    return df.sort_values(by=['scikit_cos'], ascending = False).head(10)
  return None

# lenta
def getmovietitles(df):
  titles =[]
  for idx0, row0 in df.iterrows():
    for idx1, row1 in datasets[0].iterrows():
      if row0['movieId'] == row1['movieId']:
        titles.append(row1['title'])
        break
  df['title'] = titles
  return df

# 2o. Análise em lotes
def analyse(times):
  for i in range(times):
    print("\n#### TESTE "+str(i+1)+" ####")
    # usuário deve ter mais de 10 filmes com avaliação 5
    user =  pd.DataFrame()
    while user.shape[0] < 11:
      # sorteia usuário
      id = random.randrange(1, 610)
      # filtra todos os filmes de avaliação máxima do usuário sorteado
      user = datasets[1].loc[(datasets[1]['userId'] == id) & (datasets[1]['rating'] == 5.0)]
    print("userId: "+str(id))
    # sorteia 11 filmes de avaliação máxima do usuário
    usersample = pd.DataFrame()
    usersample = sampledata(user,11,1)
    #print(usersample)
    # pega todos os ids de filmes da base original
    mids = pd.DataFrame()
    mids['movieId'] = datasets[0]['movieId']
    #print("total de filmes: "+str(mids.shape[0]))
    # pega todos os filmes do usuário
    watched = pd.DataFrame()
    watched = datasets[1].loc[(datasets[1]['userId'] == id)]
    print("filmes assistidos: "+str(watched.shape[0]))
    # primeiro exclui todos filmes do usuário do total de filmes...
    mask = mids['movieId'].isin(watched['movieId'].tolist())
    mids = mids[~mask]
    print("filmes filtrados: "+str(mids.shape[0]))
    # ...depois sorteia 10 outros filmes não assistidos dos que restaram
    mids = sampledata(mids,10,0)
    # dos 11 do usuário, sorteia 1 para servir de referência e o exclui do resto
    reference =  pd.DataFrame()
    reference = usersample.sample()
    refmovie = reference['movieId'].iloc[0]
    index = usersample[usersample['movieId'] == refmovie].index
    usersample = usersample.drop(index)
    print('movieId: '+str(refmovie))
    print('string: '+returnstring(refmovie))
    # dados de teste rotulados. 10 filmes de rating máximo mais 10 não assistidos
    testsample = mergesamples(usersample,mids)
    df = pd.DataFrame()
    # tenta as análises sequencialmente
    try:
      df = batchcompare(refmovie,testsample,'posw') 
      print("PosTags com média de máximas WuPalmer: " + str(calculataxa(df['max_rated'].tolist())))
      print(df)
    except:
      print("PosTags com média de máximas WuPalmer não funcionou.")
    try:
      df = batchcompare(refmovie,testsample,'posc')
      print("PosTags com Cosine: " + str(calculataxa(df['max_rated'].tolist())))
      print(df)
    except:
      print("PosTags com Cosine não funcionou.")
    try:
      df = batchcompare(refmovie,testsample,'wnet')
      print("Lemmas do Wordnet com Cosine: " + str(calculataxa(df['max_rated'].tolist())))
      print(df)
    except:
      print("Lemmas do Wordnet com Cosine não funcionou.")
    try:
      df = batchcompare(refmovie,testsample,'scik')
      print("Cosine do Scikit: " + str(calculataxa(df['max_rated'].tolist())))
      print(df)
    except:
      print("Cosine do Scikit não funcionou.")

def calculataxa(lista):
  c = 0
  for i in lista:
    if i == 1:
      c+=1
  return c/10

analyse(3)

#datasets[0].loc[datasets[0]['movieId'] == 1031]
#datasets[1].loc[datasets[1]['userId'] == 143]



#### TESTE 1 ####
userId: 558
filmes assistidos: 56
filmes filtrados: 9686
movieId: 3824
string: Autumn in New York (2000) Drama Romance
PosTags com média de máximas WuPalmer: 0.7
    movieId  max_rated  postag_wp
16     5012          0   0.594872
3      4308          1   0.555983
0      1222          1   0.520075
7       597          1   0.497253
8      3186          1   0.492727
1      5064          1   0.453419
9      1968          1   0.442892
14    27820          0   0.442727
10   136838          0   0.441697
5      5420          1   0.422650
PosTags com Cosine: 0.7
    movieId  max_rated  postag_cos
3      4308          1    0.400000
16     5012          0    0.365148
19    59118          0    0.316228
10   136838          0    0.258199
9      1968          1    0.223607
5      5420          1    0.223607
7       597          1    0.200000
6      5329          1    0.200000
0      1222          1    0.169031
1      5064          1    0.141421
Lemmas do Wordnet com Cosine: 0.7
  